In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import requests
import zipfile
import io
from datetime import datetime
from kafka import KafkaProducer
import time
import json
import random

In [ ]:
# Data from december 2020
r = requests.get('https://s3.amazonaws.com/tripdata/202012-citibike-tripdata.csv.zip')
  
with zipfile.ZipFile(io.BytesIO(r.content)) as ar:
    trip_data = pd.read_csv(ar.open('202012-citibike-tripdata.csv'))

In [ ]:
# Setting up the producer
producer = KafkaProducer(
  bootstrap_servers=["kafka:9092"],
  value_serializer=lambda v: json.dumps(v).encode("utf-8"))

In [ ]:
# Preprocessing 
trip_data.rename(columns = {'start station id': 'S', 
                            'end station id': 'E',
                            'birth year': 'BY',
                            'bikeid':'B',
                            'usertype':'U',
                            'gender':'G',
                            'tripduration':'D',
                            'starttime':'ST',
                            'stoptime':'ET'}, inplace = True)

columns = ['S','E','ST','ET','B','U','BY','G','D']
data_ready = trip_data[columns]

In [ ]:
data_ready.head()

In [ ]:
# Data velocity simulation
n = len(data_ready)
for i in range(0,n):

    trip = data_ready.iloc[[i]].to_dict(orient='records')
    
    producer.send(topic="citibike", value = trip[0])
    time.sleep(random.randint(0,5))